In [36]:
import pandas as pd

In [37]:
df = pd.read_csv('Datasets/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [38]:
y = df['Outcome']
X = df.drop('Outcome', axis=1)

In [39]:
from sklearn.preprocessing import StandardScaler    

In [40]:
scalar = StandardScaler()  
X = scalar.fit_transform(X) 

In [41]:
from sklearn.model_selection import train_test_split    

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)   

In [43]:
from sklearn.linear_model import LogisticRegression 
lr = LogisticRegression()   
lr.fit(X_train, y_train)    


LogisticRegression()

In [44]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, lr.predict(X_test))  

0.7532467532467533

In [107]:
import tensorflow
from tensorflow import keras
from keras import Sequential  
from keras.layers import Dense
from keras.layers import Dropout    



In [136]:
model = Sequential()    
model.add(Dense(32, input_dim=8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 

model.fit(X_train, y_train, epochs=20, batch_size=1, validation_data=(X_test, y_test))  

Epoch 1/20


c:\Users\ASUS\Miniconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


614/614 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6130 - loss: 0.6370 - val_accuracy: 0.7792 - val_loss: 0.5071
Epoch 2/20
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7840 - loss: 0.4885 - val_accuracy: 0.7662 - val_loss: 0.4919
Epoch 3/20
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7774 - loss: 0.4484 - val_accuracy: 0.7727 - val_loss: 0.4973
Epoch 4/20
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7885 - loss: 0.4711 - val_accuracy: 0.7727 - val_loss: 0.4997
Epoch 5/20
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7911 - loss: 0.4193 - val_accuracy: 0.7597 - val_loss: 0.5156
Epoch 6/20
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7493 - loss: 0.4658 - val_accuracy: 0.7662 - val_loss: 0.5185
Epoch 7/20
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8047 - loss: 0.3994 - val_accuracy: 0.7532 - val_loss: 0.5240
Epoch 8/20
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7688 - loss: 0.4547 - val_accuracy: 0.7468 - val_

In [47]:
import keras_tuner as kt 

In [129]:
def build_model(hp):
    model = Sequential()

    counter = 0

    for i in range(hp.Int('num_layers', 1, 5)):
        if counter == 0:
            model.add(Dense(units=hp.Int('units'+str(i), min_value=8, max_value=64, step=8), activation='relu', input_dim=8))
            counter += 1
        else:
            model.add(Dense(units=hp.Int('units'+str(i), min_value=8, max_value=64, step=8), activation='relu'))
            model.add(Dropout(hp.Float('dropout'+str(i), 0, 0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid', name='output'))

    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop'])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [130]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name='diabetes')

c:\Users\ASUS\Miniconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [131]:
tuner.search(X_train, y_train, epochs=5, validation_data = (X_test, y_test),verbose=1)        

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7662337422370911

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 13s


In [132]:
tuner.get_best_hyperparameters()[0].values  

{'num_layers': 3,
 'units0': 64,
 'optimizer': 'rmsprop',
 'units1': 56,
 'dropout1': 0.2,
 'units2': 64,
 'dropout2': 0.30000000000000004,
 'units3': 48,
 'dropout3': 0.1,
 'units4': 32,
 'dropout4': 0.4}

In [133]:
model=tuner.get_best_models()[0]

c:\Users\ASUS\Miniconda3\lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [134]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 56)             │         3,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 56)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         3,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,929 (30.97 KB)

 Trainable params: 7,929 (30.97 KB)

 Non-trainable params: 0 (0.00 B)

In [135]:
model.fit(X_train, y_train, epochs=50,initial_epoch=6, batch_size=1, validation_data=(X_test, y_test))  

Epoch 7/50


614/614 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7581 - loss: 0.5142 - val_accuracy: 0.7338 - val_loss: 0.5791
Epoch 8/50
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7586 - loss: 0.4994 - val_accuracy: 0.7662 - val_loss: 0.6258
Epoch 9/50
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7652 - loss: 0.6422 - val_accuracy: 0.7403 - val_loss: 0.6726
Epoch 10/50
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7840 - loss: 0.4748 - val_accuracy: 0.7273 - val_loss: 0.7245
Epoch 11/50
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8154 - loss: 0.4356 - val_accuracy: 0.7338 - val_loss: 0.6837
Epoch 12/50
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8015 - loss: 0.4707 - val_accuracy: 0.7403 - val_loss: 0.7858
Epoch 13/50
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7839 - loss: 0.5235 - val_accuracy: 0.7403 - val_loss: 0.7892
Epoch 14/50
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7998 - loss: 0.5481 - val_accuracy: 0.7403 -